In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_bronze = "bronze"
schema_silver = "silver"

In [0]:
from pyspark.sql.functions import col, when, concat, lit, initcap, trim, lower, regexp_replace, to_date

In [0]:
silver_clientes = (
    spark.table(f"{catalog_name}.{schema_bronze}.clientes_raw")
    # Convierte id_cliente a tipo bigint
    .withColumn("id_cliente", col("id_cliente").cast("bigint"))
    # Limpia y estandariza el nombre del cliente
    .withColumn(
        "nombre",
        when(col("nombre").isNull(), concat(lit("Cliente"), lit(" "), col("id_cliente").cast("string")))
        .when(col("nombre").rlike("(?i).*cliente.*"), initcap(trim(col("nombre"))))
        .otherwise(concat(initcap(trim(col("nombre"))), lit(" "), col("id_cliente").cast("string")))
    )
    # Genera el email en minúsculas usando el nombre sin espacios
    .withColumn("email", lower(concat(regexp_replace(trim(col("nombre")), " ", ""), lit("@mail.com"))))
    # Limpia y estandariza la ciudad
    .withColumn("ciudad", initcap(trim(col("ciudad"))).cast("string"))
    # Normaliza la fecha de registro en distintos formatos
    .withColumn(
        "fecha_registro",
        when(col("fecha_registro") == "N/A", None)
        .when(col("fecha_registro").rlike("^[0-9]{4}-[0-9]{2}-[0-9]{2}$"), to_date(col("fecha_registro"), "yyyy-MM-dd"))
        .when(col("fecha_registro").rlike("^[0-9]{2}-[0-9]{2}-[0-9]{4}$"), to_date(col("fecha_registro"), "MM-dd-yyyy"))
        .when(col("fecha_registro").rlike("^[0-9]{2}/[0-9]{2}/[0-9]{4}$"), to_date(col("fecha_registro"), "dd/MM/yyyy"))
        .when(col("fecha_registro").rlike("^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}$"), to_date(col("fecha_registro"), "yyyy/MM/dd HH:mm:ss"))
        .otherwise(None)
    )
    # Rellena ciudad desconocida
    .fillna("Desconocido", subset=["ciudad"])
    # Elimina duplicados por id_cliente
    .dropDuplicates(["id_cliente"])
)

In [0]:
display(silver_clientes)

In [0]:
silver_clientes.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_silver}.clientes")

In [0]:
%sql
select distinct ciudad
from dmc_tallerfinaldatabricks_mariovento.silver.clientes